# Predict from a Saved PyTorch Model
This notebook demonstrates how to load a saved model and use it to make predictions from an image.

In [3]:
import torch
from torchvision import transforms, models
from PIL import Image
import joblib

# Load the label encoder
label_encoder = joblib.load('label_encoder.joblib')

# Get the number of classes from the label encoder
num_classes = len(label_encoder.classes_)

# Create the model with the correct architecture (assuming you used ResNet50 during training)
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Load the trained model
model.load_state_dict(torch.load('mtg_card_classifier.pth'))
model.eval()

/tmp/ipykernel_65692/4182071136.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('mtg_card_classifier.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Preprocess the Image
Apply the same transformations that were used during training.

In [20]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the image
image_path = 'datasets/tcg_magic/data/test/00464232_2594_405a_95bb_f9b2a3287871/0017.png'
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0)  # Add batch dimension

## Predict the Label
Perform inference using the trained model.

In [21]:
import torch.nn.functional as F

# Make prediction
with torch.no_grad():
    outputs = model(image)
    probabilities = F.softmax(outputs, dim=1)
    confidence, predicted = torch.max(probabilities, 1)

# Get the predicted class name and confidence
predicted_class = label_encoder.inverse_transform([predicted.item()])[0]
confidence_percentage = confidence.item() * 100

print(f"Predicted class: {predicted_class}")
print(f"Confidence: {confidence_percentage:.2f}%")

Predicted class: 00464232_2594_405a_95bb_f9b2a3287871
Confidence: 39.99%


In [22]:
# Make prediction
with torch.no_grad():
    outputs = model(image)
    probabilities = F.softmax(outputs, dim=1)

# Get top 5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)
top5_prob = top5_prob.squeeze().tolist()
top5_catid = top5_catid.squeeze().tolist()

# Print top 5 predictions
print("Top 5 predictions:")
for i in range(5):
    class_name = label_encoder.inverse_transform([top5_catid[i]])[0]
    confidence_percentage = top5_prob[i] * 100
    print(f"{i+1}. {class_name}: {confidence_percentage:.2f}%")

Top 5 predictions:
1. 00464232_2594_405a_95bb_f9b2a3287871: 39.99%
2. 002f647a_25f8_461b_8617_52674f5d577c: 10.47%
3. 008c8d72_097e_472d_88c8_78bf29e42e32: 3.96%
4. 00127acd_a284_4bc8_bdfa_895914e3cd49: 2.39%
5. 00ad3531_399c_4897_b0ee_ad2a26445a17: 2.27%
